# Clustering Crypto

In [325]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [326]:
# Alternatively, use the provided csv file:
path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(path)
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1252 non-null   object 
 1   CoinName         1252 non-null   object 
 2   Algorithm        1252 non-null   object 
 3   IsTrading        1252 non-null   bool   
 4   ProofType        1252 non-null   object 
 5   TotalCoinsMined  744 non-null    float64
 6   TotalCoinSupply  1252 non-null   object 
dtypes: bool(1), float64(1), object(5)
memory usage: 60.0+ KB


### Data Preprocessing

In [327]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.drop(columns=['Unnamed: 0'])

In [328]:
# Keep only cryptocurrencies that are trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace = True)

In [329]:
# Keep only cryptocurrencies with a working algorithm
crypto_df['Algorithm'].isnull().values.any()

False

In [330]:
# Remove the "IsTrading" column
crypto_df.drop(['IsTrading'], axis = 1, inplace = True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [331]:
# Remove rows with at least 1 null value
crypto_df2.dropna(inplace = True)

In [332]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df3 = crypto_df[crypto_df['TotalCoinsMined'] > 0]

In [333]:
# Drop rows where there are 'N/A' text values
crypto_df3.dropna(how = 'all')

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [334]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = crypto_df3['CoinName']
coin_name_df.head()

0      42 Coin
2      404Coin
5    EliteCoin
7      Bitcoin
8     Ethereum
Name: CoinName, dtype: object

In [335]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df4 = crypto_df3.drop(['CoinName'], axis = 1)
crypto_df4.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [336]:
# Create dummy variables for text features & storing them on a separate DataFrame X 
dummies = pd.get_dummies(crypto_df4[['Algorithm', 'ProofType']], drop_first = True)
X = pd.concat([crypto_df4.drop(['Algorithm', 'ProofType'], axis = 1), dummies], axis = 1)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [337]:
# Standardize data from the X DataFrame using StandardScaler from sklearn
X_scaler = StandardScaler().fit_transform(X)
print(X_scaler)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Reducing Dimensions Using PCA

In [338]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components = 3)
X_pca = pca.fit_transform(X_scaler)

In [339]:
# Create a DataFrame with the principal components data
pc_df = pd.DataFrame(
    data = X_pca, columns = ['PC 1', 'PC 2', 'PC 3']
)
pc_df.head()

,PC 1,PC 2,PC 3
0,-0.328087,1.007528,-0.518295
1,-0.311402,1.007429,-0.518606
2,2.298863,1.685485,-0.653783
3,-0.144423,-1.309422,0.194364
4,-0.154539,-2.025417,0.381601


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [340]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters = i, random_state = 0)
    km.fit(pc_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow = {'k' : k, 'inertia' : inertia}
elbow_df = pd.DataFrame(elbow)
elbow_df.hvplot.line(x = 'k', y = 'inertia', xticks = k, title = 'Elbow Curve')

C:\Users\aidan\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k = 5`

In [341]:
# Initialize the K-Means model
K_model = KMeans(n_clusters = 5, random_state = 0)
# Fit the model
K_model.fit(pc_df)
# Predict clusters
predict = K_model.predict(pc_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pc_df['Class'] = K_model.labels_
pc_df.head()

,PC 1,PC 2,PC 3,Class
0,-0.328087,1.007528,-0.518295,0
1,-0.311402,1.007429,-0.518606,0
2,2.298863,1.685485,-0.653783,4
3,-0.144423,-1.309422,0.194364,1
4,-0.154539,-2.025417,0.381601,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [342]:
# Create a 3D-Scatter with the PCA data and the clusters. Including parameters on the plot: hover_name="CoinName" and hover_data=["Algorithm"]
cluster_df = pd.concat([crypto_df3, pc_df], axis = 1)
fig = px.scatter_3d(
    cluster_df,
    x = 'PC 3',
    y = 'PC 2',
    z = 'PC 1',
    hover_name = 'CoinName',
    hover_data = ['Algorithm'],
    width = 1000,
)
fig.update_layout(legend = dict(x = 0, y =1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [343]:
# Table with tradable cryptos
cluster_df.hvplot.table(columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], width = 1000)
cluster_df.dropna().head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.328087,1.007528,-0.518295,0.0
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,2.298863,1.685485,-0.653783,4.0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,-0.161254,-1.151903,0.015265,1.0
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.155362,-2.210611,0.354794,1.0
8,Ethereum,Ethash,PoW,1.076842e+08,0,-0.152979,-2.025535,0.381590,1.0


In [344]:
# Print the total number of tradable cryptocurrencies
count = cluster_df['CoinName'].count()
print(f'There is a total of {count} cryptocurrencies which are tradable')

There is a total of 532 cryptocurrencies which are tradable


#### Scatter Plot with Tradable Cryptocurrencies

In [345]:
# Scale data to create the scatter plot
cluster_df['TotalCoinSupply'] = cluster_df['TotalCoinSupply'].astype(float)/100000000
cluster_df['TotalCoinsMined'] = cluster_df['TotalCoinsMined'].astype(float)/100000000

In [346]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
cluster_df.hvplot(
    kind = 'scatter',
    x = 'TotalCoinsMined',
    y = 'TotalCoinSupply',
    hover_cols = ['CoinName'],
    by = 'Class'
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)